# Python with PostgreSQL & PostGIS

<span>Note: Please always run the complete Jupyter Notebook from the beginning, as object names such as 'sql' and 'gdf' are reused in the code cells.</span>

## Libraries and Settings

In [1]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/python_postgresql_postgis


## Create database connection

In [3]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Current database:", db_name)
except Exception as e:
    print("Error connecting to the database:", e)

# Dispose the engine
engine.dispose()

Current database: osm_switzerland


## List tables in database

In [4]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public';"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

geography_columns
geometry_columns
spatial_ref_sys
planet_osm_roads
planet_osm_line
planet_osm_polygon
planet_osm_point


## Show columns and data types of a selected table

In [5]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Dispose the engine
engine.dispose()

# Print the DataFrame
df

,column_name,data_type
0,osm_id,bigint
1,access,text
2,addr:housename,text
3,addr:street,text
4,addr:housenumber,text
...,...,...
68,width,text
69,wood,text
70,z_order,integer
71,way_area,real


## Query: Select buildings for which the full address is available in defined zip code areas

In [6]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query 
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p."addr:street" IS NOT NULL
                AND p."addr:housenumber" IS NOT NULL
                AND p."addr:city" IS NOT NULL
                AND p."addr:postcode" IN ('8001', '8002')"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


,osm_id,addr:street,addr:housenumber,addr:city,addr:postcode,building,geom
0,157954139,Löwenstrasse,67,Zürich,8001,commercial,"POLYGON ((8.53805 47.37717, 8.53808 47.37716, ..."
1,185748494,Predigerplatz,26,Zürich,8001,apartments,"POLYGON ((8.54549 47.37359, 8.54554 47.37353, ..."
2,185748492,Predigerplatz,22,Zürich,8001,yes,"POLYGON ((8.54543 47.37356, 8.54553 47.37345, ..."
3,185748488,Predigerplatz,14,Zürich,8001,apartments,"POLYGON ((8.5453 47.37352, 8.5454 47.3734, 8.5..."
4,185748487,Predigerplatz,10,Zürich,8001,apartments,"POLYGON ((8.54523 47.3735, 8.54532 47.37339, 8..."
...,...,...,...,...,...,...,...
2347,80338523,Löwenstrasse,10,Zürich,8001,yes,"POLYGON ((8.53494 47.37345, 8.53498 47.37344, ..."
2348,318684689,Nüschelerstrasse,36,Zürich,8001,yes,"POLYGON ((8.53521 47.37333, 8.53523 47.37333, ..."
2349,326259135,Uraniastrasse,32,Zürich,8001,yes,"POLYGON ((8.53541 47.37349, 8.53555 47.37343, ..."
2350,80338508,Uraniastrasse,34,Zürich,8001,yes,"POLYGON ((8.53534 47.37342, 8.53548 47.37336, ..."


## Show selected features on map

<span">Note the popup field in the map, which has been added to provide additional information about buildings.</span>

<span">Example of alternative background maps (maptiles) are:</span>
- <span">EsriWorldImagery</span>
- <span">EsriWorldTopoMap</span>
- <span">EsriWorldGrayCanvas</span>
- <span">CartoDBDarkMatter</span>
- <span">CartoDBPositron</span>


In [7]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15,
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='geojson',
    weight=0.5,
    fill_color='greenyellow',
    fillOpacity=0.8,
    popup=folium.GeoJsonPopup(fields=['addr:street',
                                      'addr:housenumber',
                                      'addr:city',
                                      'addr:postcode',
                                      'building'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select coffee stores

In [7]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            h.osm_id,
            h.shop,
            h.name,
            ST_Transform(h.way, 4326) AS geom
        FROM planet_osm_point h
        WHERE h.shop = 'coffee';"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf.head()


,osm_id,shop,name,geom
0,11017970651,coffee,Vitudurum.com GmbH,POINT (8.75393 47.50449)
1,4841545900,coffee,Nespresso Boutique,POINT (8.59574 47.40825)
2,12023129318,coffee,Atinkana,POINT (8.59894 47.40057)
3,10967856734,coffee,Beanbank,POINT (8.53654 47.36939)
4,5297627699,coffee,Rent a Barista,POINT (8.53152 47.37352)


## Show selected features on map

In [8]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'shop'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all supermarkets in a distance of 1000m around the central station in the city of Winterthur.

<span>Note:</span>

<span>For each supermarket, the distance to the central station in meters is calculated and stored as new column 'distance_meters'.</span>

<span>In addition, a popup field was added to the map, allowing users to view detailed information about each selected feature when they click on it.</span>

<span>The WGS84 (World Geodetic System 1984) coordinates in ST_MakePoint(LON, LAT) were derived from: https://tools.retorte.ch/map.</span>


In [9]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            p.osm_id,
            p.shop,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'supermarket'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography,
                1000
            )
        ORDER BY distance_meters;"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf.head()


,osm_id,shop,name,distance_meters,geom
0,706203439,supermarket,Coop,159.883419,POINT (8.72594 47.50085)
1,4109460421,supermarket,Asia Shop,162.391281,POINT (8.72208 47.50101)
2,3831772784,supermarket,Migros,247.578208,POINT (8.72115 47.49916)
3,7380954145,supermarket,Alnatura,256.838011,POINT (8.72074 47.49958)
4,4095400190,supermarket,ALDI,274.275393,POINT (8.72476 47.4979)


## Show selected features on map

In [10]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'distance_meters'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all roads classified as 'motorway' and create a 5000m buffer around these roads.

In [11]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query (major roads)
sql = """-- Create buffer around major roads
        SELECT 
            1 as group_id,
            ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 2000)), 4326) AS geom
        FROM public.planet_osm_roads AS p
        WHERE
            highway = 'motorway';"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Dispose the engine
engine.dispose()

## Show selected features on map

In [12]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [13]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-09-24 17:12:28
Python Version: 3.12.1
-----------------------------------
